In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.insert(0,'/home/daniel/code/dosorio79/code/kolapally/computer_vision')

In [13]:
from compvis.preprocess.face_detect_single import face_detect_single

# Single image 

In [15]:
crop_path, crop_coord = face_detect_single('/home/daniel/code/dosorio79/code/kolapally/computer_vision/notebooks/DSO_face-detect_haar/img/smokey.png', 'png')

1/1 [==============================] - 0s 320ms/step
/home/daniel/code/dosorio79/code/kolapally/computer_vision/compvis/interface/cropped/image_face0.png is saved
/home/daniel/code/dosorio79/code/kolapally/computer_vision/compvis/interface/cropped/image_face1.png is saved


In [16]:
crop_coord

{'face0': (873, 277, 169, 222), 'face1': (556, 45, 195, 257)}

# Multiple files